# Introduction to Automatic Speech Recognition

**Name**: 

This lab is an introduction to automatic speech recognition (ASR). In a nutshell, the ASR process consists of the following blocks:

1. **Feature extraction**: The audio waveform is preprocessed into normalized features such as spectrogram or MFCCs.
2. **Acoustic model**: The features are transformed into high-dimensional embeddings via a deep neural network (here: [wav2vec2](https://arxiv.org/pdf/2006.11477)), and then a classifier is applied to predict the class of each token (=character) at each time step.
3. **Decoding**: A transcript is generated from the sequence of class probabilities, either using a language model or not.

<center><a href="https://developer.nvidia.com/blog/how-to-build-domain-specific-automatic-speech-recognition-models-on-gpus/">
    <img src="https://developer-blogs.nvidia.com/wp-content/uploads/2019/12/ASR-Pipeline-1.png" width="600"></a></center>
    
In this lab, we experiment with ASR using pretrained acoustic models (which include feature extraction for simplicity). We provide a dataset of real-life speech audio (with trancription) in order to study the influence of several factors onto the performance of a modern ASR system. We focus on the usage of the acoustic model, thus we use a simple decoder with no language model (this will be the topic of the next lab).

This series of lab relies on [torchaudio](https://pytorch.org/audio/stable/index.html), which is a pytorch-based librairy for audio/signal processing. More specifically, this lab is based on this [tutorial](https://pytorch.org/audio/stable/tutorials/speech_recognition_pipeline_tutorial.html#sphx-glr-tutorials-speech-recognition-pipeline-tutorial-py), which you are encouraged to check.
 

In [ ]:
import torch
import torchaudio
import IPython
import os
import fnmatch
import matplotlib.pyplot as plt
torch.random.manual_seed(0);

MAX_FILES = 100 # lower this number for processing a subset of the dataset

In [ ]:
# Main dataset path - If needed, you can change it HERE but NOWHERE ELSE in the notebook!
data_dir = "../dataset/"

In [ ]:
# Speech and transcripts sub-directories paths
data_speech_dir = os.path.join(data_dir, 'speech')
data_transc_dir = os.path.join(data_dir, 'transcription')

## Load and display an audio example

Let us first load an audio file, listen to it using an audio reader, and plot the waveform.

In [ ]:
# Example file
audio_file = '61-70968-0001.wav'
audio_file_path = os.path.join(data_speech_dir, audio_file)
print(f"Audio file path: {audio_file_path}")

waveform, sr = torchaudio.load(audio_file_path, channels_first=True)
IPython.display.Audio(data=waveform, rate=sr)

In [ ]:
# Plot the speech signal waveform
plt.figure()
xt = torch.arange(waveform.size(1)) / sr
plt.plot(xt, waveform.T)
plt.yticks([])
plt.xlabel("Time (s)")
plt.show()

## Acoustic model

Let us create an acoustic model that performs feature extraction and classification. In this lab, we use [wav2vec2](https://arxiv.org/pdf/2006.11477), a self-supervised audio model that produces state-of-the-art results for many audio tasks (including ASR).

For convenience, both the model definition and pretrained wav2vec2 weights can be directly loaded using torchaudio, which includes two types of weights:
- The pretrained weights [without fine-tuning](https://pytorch.org/audio/stable/pipelines.html#id3), that can be fine-tuned for any other downstream tasks (e.g., emotion recognition or language detection).
- The pretrained weights that are [fine-tuned for the ASR task](https://pytorch.org/audio/stable/pipelines.html#id36), using an extra classification layer after feature extraction with wav2vec2.

In this series of lab, we use the latter. Below we provide the code for constructing a model and fetching the pretrained weights along with the classification labels. Here we use the `WAV2VEC2_ASR_BASE_100H` model. As the name indicates, it is a basic wav2vec2 architecture that is fine-tuned for ASR using 100 hours of the Librispeech dataset (containing both speech and transcriptions). Other models can be fetched using torchaudio (see the list [here](https://pytorch.org/audio/stable/pipelines.html#id36)).

In [ ]:
# Load the acoustic model
model_name = 'WAV2VEC2_ASR_BASE_100H'
bundle = getattr(torchaudio.pipelines, model_name)
acoustic_model = bundle.get_model()

In [ ]:
# Display the details of the model's architecture (more info on this in the Neural Network labs!)
print(acoustic_model)

In [ ]:
# Get and display the classification labels
labels = bundle.get_labels()
print('Labels:', labels)

The labels are the characters that can be predicted by the acoustic model. Note that there are two special labels:
- `-` is the blank label, which corresponds to no character.
- `|` is the end-of-word label, which is useful to further segment a list of characters into a sequence of words.

We can now apply the acoustic model to the example audio signal wa have loaded above. We can either compute the acoustic features or predicting the labels after the classification layer (also called the *emission* matrix/tensor):
- `features` is a list of tensors, where each tensor is the output of a transformer layer from the wav2vec2 model.

In [ ]:
with torch.inference_mode():
    features, _ = acoustic_model.extract_features(waveform)

print(f"Number of transformer layers: {len(features)}")
print(features[0].shape)

- `emission` is a tensor of size `[1, time steps, number of classes]` corresponding to the output of the classification layer (`1` corresponds to the batch size). Note that it is in the form of logits, not probabilities (thus it is not normalized in $[0, 1]$).

In [ ]:
with torch.inference_mode():
    emission, _ = acoustic_model(waveform)

print(emission.shape)

In [ ]:
# Vizualize the result of the classification
plt.figure()
plt.imshow(emission[0].cpu().T)
plt.title("Classification result")
plt.xlabel("Frame (time-axis)")
plt.ylabel("Class")
plt.tight_layout()
plt.show()

We can compute the most likely label at each time frame to get a first rough transcript estimate, as done below.

In [ ]:
indices = torch.argmax(emission[0], dim=-1)  # take the most likely index at each time step
pred_labels = [labels[i] for i in indices] # transform each index into the corresponding label
print(pred_labels)

Note that there are many blank labels `'-'`, as well as duplicates. This is why some further postprocessing / decoding is needed.

## Decoding

From the emission tensor, which is the sequence of probabilities (or logits) for each class / token, we now want to generate a transcript (or *hypothesis*). This process is called *decoding*. Decoding can be complex because it implies accounting for the context in order to avoid duplicates, assembling characters into words, identifying precise word start / end, and building sequences of words that make sense given some underlying language model.

Such refined decoding techniques involve using external ressources such as lexicon and language models. In this lab however, we rely on a naive approach called a *greedy* decoder, which does not depend on such components. The context information is not used (= we pick the best hypothesis at each time step), and only one transcript is generated by applying some basic post-processing (removing blank tokens and duplicates).

In [ ]:
class GreedyDecoder(torch.nn.Module):
    def __init__(self, labels, blank_token_indx=0):
        super().__init__()
        self.labels = labels
        self.blank_token_indx = blank_token_indx

    def forward(self, emission):
        """Given a sequence emission over labels, decode the transcript
        Args:
          emission (Tensor): Logit tensors. Shape `[1, num_seq, num_label]`.
        Returns:
          transcript (List of strings): The resulting transcript
        """
        emission = emission[0] # take the first element in the batch (only one)
        indices = torch.argmax(emission, dim=-1)  # take the most likely index at each time step
        indices = torch.unique_consecutive(indices, dim=-1) # remove duplicates
        indices = [i for i in indices if i != self.blank_token_indx] # remove blank token
        transcript = "".join([self.labels[i] for i in indices]) # convert indices into tokens
        transcript = transcript.replace("|", " ").lower().split() # a bit of post-processing
        return transcript

In [ ]:
# Instanciate the decoder, apply it, and display the result
decoder = GreedyDecoder(labels=labels)
est_transcript = decoder(emission)
print(est_transcript)

## Evaluation

To evaluate the quality of the transcript above, we need to compare it to some reference. More precisely, we can compute the [word error rate](https://en.wikipedia.org/wiki/Word_error_rate) (WER) between the reference and estimated transcripts. The WER is a measure of speech recognition performance expressed in % (lower is better) based on the Levenshtein distance, which allows to account for the fact that the true and estimated transcripts might have different lengths; more info in the course.

<span style="color:red"> **Exercise 1**</span>. Define the path to the true transcript file - the transcript file name is the same as the audio, except for the extension (`.txt` instead of `.wav`). Load the content of this file to get the true transcript, and preprocess it to the same format as the estimated transcript above (i.e., use the `.lower()` and `.split()` methods). Display the true transcript. Then, using the provided `get_wer` function, compute and display the WER.

In [ ]:
def get_wer(true_transcript, est_transcript):
    wer = torchaudio.functional.edit_distance(true_transcript, est_transcript) / len(true_transcript)
    return wer

In [ ]:
def get_true_transcript(transc_file_path):
    with open(transc_file_path, "r") as f:
        true_transcript = f.read()
    true_transcript = true_transcript.lower().split()
    return true_transcript
    
# Load and display the true transcription
transc_file_path = os.path.join(data_transc_dir, audio_file.replace('wav', 'txt'))
true_transcript = get_true_transcript(transc_file_path)
print(true_transcript)

In [ ]:
# Compute and display the WER score
wer = get_wer(true_transcript, est_transcript)
print(f"WER: {wer*100:.1f} %")

## Process a folder

<span style="color:red"> **Exercise 2**</span>. Process the entire dataset folder (=the 100 files) using the wav2vec2 acoustic model and the greedy decoder above, and compute and display the mean WER over the dataset.

In [ ]:
# We provide a function below that returns the list of all files in a directory
def find_files(directory, pattern='*.wav'):
    """Recursively finds all files matching the pattern."""
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, pattern):
            files.append(filename)
    files = sorted(files)
    return files

# Get the list of audio files in the asr dataset
audio_files = find_files(data_speech_dir)
print(audio_files[:10])

In [ ]:
# Function to process a folder
def process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=True, max_files=None):

    # Get the list of files in the dataset folder
    audio_files = find_files(data_speech_dir)

    # Take a subset of files
    nfiles = len(audio_files)
    if max_files:
        nfiles = min(nfiles, max_files)
    audio_files = audio_files[:nfiles]

    # Initialize lists containing true and estimated transcripts, as well as WER
    true_transcript_all = []
    est_transcript_all = []
    wer_all = []

    for iaf, audio_file in enumerate(audio_files):
        
        # Get files path (speech and transcript)
        audio_file_path = os.path.join(data_speech_dir, audio_file)
        transc_file_path = os.path.join(data_transc_dir, audio_file.replace('wav', 'txt'))

        # Load an audio signal
        waveform, sr = torchaudio.load(audio_file_path, channels_first=True)

        # Apply acoustic model and decoder
        with torch.inference_mode():
            emission, _ = acoustic_model(waveform)
            est_transcript = decoder(emission)
            
        # Load the true transcription
        true_transcript = get_true_transcript(transc_file_path)
        
        # Compute WER
        wer = get_wer(true_transcript, est_transcript)
        wer_all.append(wer)

        est_transcript_all.append(est_transcript)
        true_transcript_all.append(true_transcript)
        
        # Display results
        if verbose:
            print(f"File {iaf+1} / {nfiles}")
            print('Estimated transcript: ', est_transcript)
            print('True transcript: ', true_transcript)
            print(f"WER: {wer*100} %")

    wer_mean = torch.FloatTensor(wer_all).mean()

    return wer_mean, est_transcript_all, true_transcript_all

In [ ]:
# Process the dataset and display the mean WER
wer_mean, _, _ = process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=False, max_files=MAX_FILES)
print(f"Mean WER: {wer_mean*100:.1f} %")

## Influence of the noise

ASR systems are expected to be sensitive to the noise: while a model might work well on a clean recording, its performance will drop in real-life conditions where speech is contaminated with noise. We can simulate such a scenario by adding white noise to our speech waveforms.

The amount of noise is controlled by the *signal-to-noise ratio* (SNR), expressed in dB. The lower the SNR, the noisier the mixture: a clean speech signal corresponds to SNR=+$\infty$, while a pure noise signal corresponds to SNR=-$\infty$.

In [ ]:
# The functions below allow to generate white noise, and add it to the clean speech at a certain SNR level

def adjust_noise_snr(
    speech,
    noise,
    snr_dB,
    silence_threshold=0.01,
):

    # Filter out silence from speech signal
    speech_wo_silence = speech[torch.abs(speech) > silence_threshold]

    # Calculate the total energy of the speech and noise signals
    speech_power = torch.sum(torch.square(speech_wo_silence))
    noise_power = torch.sum(torch.square(noise))

    # Compute gain factor to achieve desired SNR
    gain_factor = torch.sqrt((speech_power / noise_power) * 10 ** (-snr_dB / 10))

    # Scale the noise accordingly
    scaled_noise = gain_factor * noise

    return scaled_noise


def add_noise(speech, snr_dB=0):
    # Generate random white noise
    noise = torch.randn_like(speech)
    # Adjust noise level
    noise = adjust_noise_snr(speech, noise, snr_dB)
    # Add the noise to the clean speech
    speech_noisy = speech + noise
    return speech_noisy, noise

In [ ]:
# Add noise to the original audio signal at SNR = 0 dB
speech_noisy_0dB, noise_OdB = add_noise(waveform, snr_dB=0)
IPython.display.Audio(data=speech_noisy_0dB, rate=sr)

In [ ]:
# Add noise to the original audio signal at SNR = 5 dB
speech_noisy_5dB, noise_5dB = add_noise(waveform, snr_dB=5)
IPython.display.Audio(data=speech_noisy_5dB, rate=sr)

In [ ]:
# Display the noisy signals (vs. the clean one)
plt.figure()
xt = torch.arange(waveform.size(1)) / sr
plt.plot(xt, speech_noisy_0dB.T, label='noisy, SNR=0 dB')
plt.plot(xt, speech_noisy_5dB.T, label='noisy, SNR=5 dB')
plt.plot(xt, waveform.T, label='clean')
plt.yticks([])
plt.xlabel("Time (s)")
plt.legend()
plt.show()

<span style="color:red"> **Exercise 3**</span>. For each SNR $\in [-5, 0, 5, 10]$ dB, add noise to the files in the dataset, perform ASR and compute/display the mean WER. What do you observe?

In [ ]:
# Process a folder
def process_folder_noisy(data_speech_dir, data_transc_dir, acoustic_model, decoder, snr_dB=0, verbose=True, max_files=None):

    # Get the list of files in the dataset folder
    audio_files = find_files(data_speech_dir)

    # Take a subset of files
    nfiles = len(audio_files)
    if max_files:
        nfiles = min(nfiles, max_files)
    audio_files = audio_files[:nfiles]

    # Initialize lists containing true and estimated transcripts, as well as WER
    true_transcript_all = []
    est_transcript_all = []
    wer_all = []

    for iaf, audio_file in enumerate(audio_files):
        
        # Get files path
        audio_file_path = os.path.join(data_speech_dir, audio_file)
        transc_file_path = os.path.join(data_transc_dir, audio_file.replace('wav', 'txt'))

        # Load an audio signal
        waveform, sr = torchaudio.load(audio_file_path, channels_first=True)

        # Add noise
        waveform, _ = add_noise(waveform, snr_dB)
        
        # Apply acoustic model and decoder
        with torch.inference_mode():
            emission, _ = acoustic_model(waveform)
            est_transcript = decoder(emission)
            
        # Load the true transcription
        true_transcript = get_true_transcript(transc_file_path)
        
        # Compute WER
        wer = get_wer(true_transcript, est_transcript)
        wer_all.append(wer)

        est_transcript_all.append(est_transcript)
        true_transcript_all.append(true_transcript)
        
        # Display results
        if verbose:
            print(f"File {iaf+1} / {nfiles}")
            print('Estimated transcript: ', est_transcript)
            print('True transcript: ', true_transcript)
            print(f"WER: {wer*100} %")

    wer_mean = torch.FloatTensor(wer_all).mean()

    return wer_mean, est_transcript_all, true_transcript_all

In [ ]:
# Perform ASR at various SNR levels
snr_list = [-5, 0, 5, 10]

for snr_dB in snr_list:
    wer_mean, _, _ = process_folder_noisy(data_speech_dir, data_transc_dir, acoustic_model, decoder, snr_dB=snr_dB, verbose=False, max_files=MAX_FILES)
    print(f"SNR level: {snr_dB} dB ---- Mean WER: {wer_mean*100:.1f} %")

**Comment**: We observe that when the level of noise increases (=lower SNR), the ASR performance decreases (=higher WER). This is in line with our expectation, since when speech is contaminated with noise, it becomes harder to understand. This further motivates to build ASR systems that are robust to noise, e.g., via applying a prior *speech enhancement* algorithm to clean up the signal / remove the noise, or training the ASR system on noisy signals directly, so it learn to process the noise in an end-to-end fashion.

## Influence of the amount of training data

So far, we have used a model that was fine-tuned for ASR on 100 hours of speech, but alternative models exist in torchaudio (recall the list of pipelines is [here](https://pytorch.org/audio/stable/pipelines.html#id36)).

<span style="color:red"> **Exercise 4**</span>. Evaluate the impact of the amount of data for fine-tuning. To that end, load models that use the same architecture (= basic wav2vec), but fine-tuned with more or loss speech data, perform ASR on the dataset, and compute the WER. What do you observe?

In [ ]:
model_list = ['WAV2VEC2_ASR_BASE_10M',
              'WAV2VEC2_ASR_BASE_100H',
              'WAV2VEC2_ASR_BASE_960H']

for model_name in model_list:
    bundle = getattr(torchaudio.pipelines, model_name)
    acoustic_model = bundle.get_model()
    decoder = GreedyDecoder(labels=bundle.get_labels())
    wer = process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=False, max_files=MAX_FILES)[0]
    print(f"Model: {model_name} --- WER: {wer*100:.1f} %")

**Comment**: Quite intuitively, the more data for fine-tuning, the better the performance. This highlights that while using a powerful word embedding system (wav2vec2) is important, performing some fine-tuning for a specific task is also crucial. 

Finally, note that many other pretrained models are readily available in torchaudio, such as a:
- larger models (e.g., `'WAV2VEC2_ASR_LARGE_10M'`)
- model trained using another dataset (e.g., `'WAV2VEC2_ASR_LARGE_LV60K_10M'`)
- alternative architectures (e.g., `'HUBERT_ASR_LARGE'`).

Feel free to experiment with these!

In [ ]:
# Different training dataset / model architecture
#model_list = ['WAV2VEC2_ASR_LARGE_LV60K_10M', # dataset (LV60k)
#              'WAV2VEC2_ASR_LARGE_LV60K_100H',
#              'WAV2VEC2_ASR_LARGE_LV60K_960H',
#              'WAV2VEC2_ASR_LARGE_10M', # large wav2vec2
#              'WAV2VEC2_ASR_LARGE_100H',
#              'WAV2VEC2_ASR_LARGE_960H',
#              'HUBERT_ASR_LARGE', # alt architecture to be compared in terms of dataset with WAV2VEC2_ASR_LARGE_LV60K_960H
#             ]

#for model_name in model_list:
#    bundle = getattr(torchaudio.pipelines, model_name)
#    acoustic_model = bundle.get_model()
#    decoder = GreedyDecoder(labels=bundle.get_labels())
#    wer = process_folder(data_speech_dir, data_transc_dir, acoustic_model, decoder, verbose=False, max_files=100)[0]
#    print(f"Model: {model_name} --- WER: {wer*100} %")

In [ ]:
## To remove; it's just a backup using 'speechbrain'

#from speechbrain.inference.ASR import EncoderDecoderASR
#model_name = "asr-crdnn-rnnlm-librispeech"
#asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/" + model_name,
#                                           savedir=model_dir + model_name)
#def compute_transcription(asr_model, signal, sr):
#    # Normalize it as for inputing to the ASR model
#    signal_norm = asr_model.audio_normalizer(signal, sr)
#    # Fake a batch
#    batch = signal_norm.unsqueeze(0)
#    # Compute the transcription with the model
#    predicted_words, _ = asr_model.transcribe_batch(
#        batch, torch.tensor([1.0])
#    )
#    return predicted_words[0]
#    
#transcribed_speech = compute_transcription(asr_model, speech, sr)
#print(transcribed_speech)